In [5]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import time
import collections

np.set_printoptions(precision=2, linewidth=200)

%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
from utils.reader import europarl_raw_data

In [7]:
from utils.attn_graph import AttentionModel

In [8]:
def show_dict_contents(d):
    for k, v in d.items():
        for k2, v2 in v.items():
            print('{0: <30}: type={1: <30}{2}{3}'.format(
                '{0}.{1}'.format(k, k2),
                str(type(v2)),
                ' shape={0}'.format(v2.shape) if isinstance(v2, np.ndarray) else '',
                ' len={0}, contents type={1}'.format(
                    len(v2),
                    type(v2[0])
                ) if isinstance(v2, list) else '',
            ))

In [9]:
def unvectorize_sentence(sentence, idx2word):
    return ' '.join([idx2word[i] 
                     for i in sentence
                     if idx2word[i] != '<bos>' and idx2word[i] != '<eos>' and idx2word[i] != '<pad>'
                    ])

In [10]:
def sample_group(data, group_num, num_samples=10):
    samples = np.random.choice(len(data['train']['X'][group_num]), size=num_samples)
    for sample in samples:
        print(unvectorize_sentence(data['train']['X'][group_num][sample], data['vocab']['lang1_idx2word']))
        print(unvectorize_sentence(data['train']['y'][group_num][sample], data['vocab']['lang2_idx2word']))
        print()

In [11]:
data = europarl_raw_data()
show_dict_contents(data)
print(
    [(x.shape, y.shape) for x, y in sorted(zip(data['train']['X'], data['train']['y']), key=lambda t: t[0].shape[1]) if x.shape[1] > 2]
)

vocab.lang1_idx2word          : type=<class 'list'>                 len=93800, contents type=<class 'str'>
vocab.lang1_word2idx          : type=<class 'dict'>                
vocab.lang2_idx2word          : type=<class 'list'>                 len=44248, contents type=<class 'str'>
vocab.lang2_word2idx          : type=<class 'dict'>                
train.X                       : type=<class 'list'>                 len=4, contents type=<class 'numpy.ndarray'>
train.y                       : type=<class 'list'>                 len=4, contents type=<class 'numpy.ndarray'>
val.X                         : type=<class 'numpy.ndarray'>        shape=(160000, 604)
val.y                         : type=<class 'list'>                 len=160000, contents type=<class 'str'>
test.X                        : type=<class 'numpy.ndarray'>        shape=(160209, 640)
test.y                        : type=<class 'list'>                 len=160209, contents type=<class 'str'>
[((63624, 8), (63624, 32)), ((29

In [12]:
sample_group(data, 0, num_samples=5)

Darum geht es .
That is what this is all about .

Das ist seine Verantwortung .
That is his responsibility .

Damit ist der Tagesordnungspunkt beendet .
That concludes the item .

Dabei sehe ich zwei Möglichkeiten .
I see two possibilities .


If we manage to do that we will be on the right road !



In [13]:
data['train']['X'][1].shape

(377096, 24)

In [15]:
tf.reset_default_graph()
model = AttentionModel(data['vocab']['lang1_idx2word'], data['vocab']['lang2_idx2word'], 128, 512)
all_graphs = model.make_all_graphs(16, data['train']['X'], data['train']['y'])
eval_graph = model.make_eval_graph(16, 30, 32, data['vocab']['lang2_word2idx']['<bos>'])
writer = tf.summary.FileWriter(logdir='logs', graph=tf.get_default_graph())
writer.flush()

In [16]:
show_dict_contents(all_graphs[0]['inputs_and_outputs'])

placeholders.inputs           : type=<class 'tensorflow.python.framework.ops.Tensor'>
placeholders.targets          : type=<class 'tensorflow.python.framework.ops.Tensor'>
placeholders.learning_rate    : type=<class 'tensorflow.python.framework.ops.Tensor'>
placeholders.max_norm         : type=<class 'tensorflow.python.framework.ops.Tensor'>
outputs.loss                  : type=<class 'tensorflow.python.framework.ops.Tensor'>
outputs.num_correct_predictions: type=<class 'tensorflow.python.framework.ops.Tensor'>
train_ops.train_op            : type=<class 'tensorflow.python.framework.ops.Operation'>
train_ops.gradient_global_norm: type=<class 'tensorflow.python.framework.ops.Tensor'>


In [17]:
show_dict_contents(eval_graph)

placeholders.inputs           : type=<class 'tensorflow.python.framework.ops.Tensor'>
outputs.outputs               : type=<class 'tensorflow.python.framework.ops.Tensor'>
outputs.attention_weights     : type=<class 'tensorflow.python.framework.ops.Tensor'>


In [26]:
data['train']['X'][3].shape

(241295, 32)

In [33]:
np.set_printoptions(precision=2, linewidth=250)

In [34]:
eval_outputs['attention_weights'][0]

array([[ 0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03],
       [ 0.04,  0.04,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03],
       [ 0.04,  0.04,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03],
       [ 0.05,  0.04,  0.04,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03],
       [ 0.05,  0.04,  0.04,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.03,  0.0

In [35]:
INITIAL_LR=5e0
MAX_NORM=1
with tf.Session() as sess:
    run_id = time.time()
    writer = tf.summary.FileWriter('logs/{0}'.format(run_id), sess.graph)
    coord = tf.train.Coordinator()
    sess.run(tf.global_variables_initializer())
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)

    learning_rate = INITIAL_LR
    max_norm = MAX_NORM
    training_outputs = {
        **all_graphs[3]['inputs_and_outputs']['outputs'],
        **all_graphs[3]['inputs_and_outputs']['train_ops'],
    }
    for i in range(235 * 50):
        for j in range(100):
            start_idx = 0 #((i % 235) * 100 + j) * 16
            end_idx = 16 #((i % 235) * 100 + j+1) * 16
            inputs = data['train']['X'][3][start_idx:end_idx, 1:-1]
            targets = data['train']['y'][3][start_idx:end_idx]
            outputs = sess.run(
                training_outputs,
                feed_dict={
                    all_graphs[3]['inputs_and_outputs']['placeholders']['inputs']: inputs,
                    all_graphs[3]['inputs_and_outputs']['placeholders']['targets']: targets,
                    all_graphs[3]['inputs_and_outputs']['placeholders']['learning_rate']: learning_rate, # * (10.0 / (10.0 + np.sqrt(i))),
                    all_graphs[3]['inputs_and_outputs']['placeholders']['max_norm']: max_norm,
                }
            )
#         samples = np.random.choice(len(data['val']['X']), size=16)
#         eval_inputs = data['val']['X'][samples][:,1:31]
#         correct_outputs = [data['val']['y'][sample] for sample in samples]
#         eval_outputs = sess.run(
#             eval_graph['outputs']['outputs'],
#             feed_dict={
#                 eval_graph['placeholders']['inputs']: eval_inputs,
#             }
#         )
        samples = np.arange(16)
        eval_inputs = data['train']['X'][3][samples][:,1:31]
        correct_outputs = [
            unvectorize_sentence(data['train']['y'][3][sample], data['vocab']['lang2_idx2word'])
            for sample in samples
        ]
        eval_outputs = sess.run(
            eval_graph['outputs'],
            feed_dict={
                eval_graph['placeholders']['inputs']: eval_inputs,
            }
        )
        print('-' * 40)
        print('i = {0}'.format(i))
        print(outputs)
        for sample_idx, sample in enumerate(samples):
            print(unvectorize_sentence(eval_inputs[sample_idx], data['vocab']['lang1_idx2word']))
            print(correct_outputs[sample_idx])
            print(unvectorize_sentence(eval_outputs['outputs'][sample_idx], data['vocab']['lang2_idx2word']))
            print(eval_outputs['attention_weights'][sample_idx])
            print()
        print('-' * 40)
            

    # Bookkeeping        
    writer.close()
    coord.request_stop()
    coord.join(threads)
    
print(outputs)

----------------------------------------
i = 0
{'loss': 4.2203517, 'num_correct_predictions': 177, 'train_op': None, 'gradient_global_norm': 2.1727002}
Ich meine , daß der Grundsatz der relativen Stabilität einen elementaren Rechtsgrundsatz der gemeinsamen Fischereipolitik darstellt und ein Vorschlag , diesen zu unterlaufen , rechtlich unzulässig wäre .
I believe that the principle of relative stability is a fundamental legal principle of the common fisheries policy and a proposal to subvert it would be legally inadmissible .
I I would in on I I would in on I I would in on I I would in on I I would in on I I would in on I I
[[ 0.03  0.03  0.03  0.03  0.03  0.03  0.03  0.03  0.03  0.03  0.03  0.03  0.03  0.03  0.03  0.03  0.03  0.03  0.03  0.03  0.03  0.03  0.03  0.03  0.03  0.03  0.03  0.03  0.03  0.03]
 [ 0.04  0.04  0.04  0.03  0.03  0.03  0.03  0.03  0.03  0.03  0.03  0.03  0.03  0.03  0.03  0.03  0.03  0.03  0.03  0.03  0.03  0.03  0.03  0.03  0.03  0.03  0.03  0.03  0.03  0.03]
 [

KeyboardInterrupt: 